## Step 0: Import Python packages

Let start by importing some basic packages 

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder


## Step 1: Download BigQuery data to our notebook


Click on the home button of you cloud console and fill in the values for your Project Number and Project ID. We are going to use te BigQuery Python SDK to query our Data Warehouse for the Natality dataset

In [2]:
import os

from google.cloud import bigquery

# Find your project number 
PROJECT_NUMBER = '697349885677'
PROJECT_ID = 'vertex-ai-demo-351514'
client = bigquery.Client(project=PROJECT_NUMBER)

Using the python SDK, we can use SQL to retrieve our dataset and send it to a python data frame

In [6]:
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = client.query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,7.063611,True,32,1,37.0
1,4.687028,True,30,3,33.0
2,7.561856,True,20,1,39.0
3,7.561856,True,31,1,37.0
4,7.312733,True,32,1,40.0


In [8]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks
count,9997.000000,10000.000000,10000.000000,9886.000000
mean,7.303938,27.226500,1.033500,38.700890
std,1.313289,6.194646,0.192825,2.538734
min,0.595248,13.000000,1.000000,18.000000
25%,6.624891,22.000000,1.000000,38.000000
50%,7.374463,27.000000,1.000000,39.000000
75%,8.126239,32.000000,1.000000,40.000000
max,12.625874,52.000000,4.000000,47.000000


In [5]:
df['is_male'].value_counts()


True     5096
False    4904
Name: is_male, dtype: int64

## Step 2: Prepare the dataset for training

In order to save on time and go through the lab, we are going to do some very light preprocessing.

In [9]:
# drop nulls
df = df.dropna()

# shuffle is a module from sklearn utils 
df = shuffle(df, random_state=2) 

# grab our target column
labels = df['weight_pounds']

# grab our data
data = df.drop(columns=['weight_pounds'])

# one hot encoding
data['is_male'] = data['is_male'].astype(int)

In [11]:
data.head()

,is_male,mother_age,plurality,gestation_weeks
40,0,18,1,39.0
5976,0,22,1,47.0
6902,0,29,1,38.0
4838,0,32,1,37.0
8335,1,29,1,39.0


## Step 3: Split the data into train test sets

Once the data is prepped, the next step is to perform our train test split and define our model. We will be creating a Sequential Kera model with an RELU activation function

In [12]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)

## Step 4: Train the model

Once we have defined the model, next step is to compile and train

In [14]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])

In [15]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1)


Epoch 1/10
209/209 [==============================] - 2s 3ms/step - loss: 4.9140 - mae: 1.2787 - mse: 4.9140 - val_loss: 1.2099 - val_mae: 0.8555 - val_mse: 1.2099
Epoch 2/10
209/209 [==============================] - 0s 2ms/step - loss: 1.4287 - mae: 0.9333 - mse: 1.4287 - val_loss: 1.9655 - val_mae: 1.1364 - val_mse: 1.9655
Epoch 3/10
209/209 [==============================] - 0s 2ms/step - loss: 1.3875 - mae: 0.9178 - mse: 1.3875 - val_loss: 1.7498 - val_mae: 1.0600 - val_mse: 1.7498
Epoch 4/10
209/209 [==============================] - 0s 2ms/step - loss: 1.3527 - mae: 0.9072 - mse: 1.3527 - val_loss: 1.3415 - val_mae: 0.9090 - val_mse: 1.3415
Epoch 5/10
209/209 [==============================] - 0s 2ms/step - loss: 1.3309 - mae: 0.8916 - mse: 1.3309 - val_loss: 1.3214 - val_mae: 0.9046 - val_mse: 1.3214
Epoch 6/10
209/209 [==============================] - 0s 2ms/step - loss: 1.3293 - mae: 0.8931 - mse: 1.3293 - val_loss: 1.1714 - val_mae: 0.8472 - val_mse: 1.1714
Epoch 7/10
209/2

## Step 5: Evaluate the model 

In [ ]:
We can grab a set of samples from our test dataset to run a quick evaluation on our predictions.

In [16]:
num_examples = 10
predictions = model.predict(x_test[:num_examples])

1/1 [==============================] - 0s 113ms/step


In [17]:
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ',y_test.iloc[i])
    print()

Predicted val:  7.488137
Actual val:  7.31273323054

Predicted val:  7.8713775
Actual val:  7.06140625186

Predicted val:  7.5233426
Actual val:  6.1244416383599996

Predicted val:  9.002631
Actual val:  6.8122838958

Predicted val:  7.694968
Actual val:  8.12623897732

Predicted val:  6.535587
Actual val:  8.000575487979999

Predicted val:  7.077868
Actual val:  7.936641432

Predicted val:  7.5172105
Actual val:  5.56226287026

Predicted val:  7.338104
Actual val:  7.62578964258

Predicted val:  8.127029
Actual val:  8.811876612139999



In [18]:
model

## Step 6: Register and Deploy Model

In [ ]:
import time
t = time.localtime()
current_time = time.strftime("%H-%M-%S", t)
print(current_time)

We are going to export our model artifact to our cloud storage bucket location. The artifacts will be stored in the export_dir variable

In [22]:
export_dir = '{}/export/natality_{}'.format('gs://'+PROJECT_ID, time.strftime("%Y%m%d-%H%M%S"))
print('Exporting to {}'.format(export_dir))
tf.saved_model.save(model, export_dir)

Exporting to gs://vertex-ai-demo-351514/export/natality_20220607-221449
INFO:tensorflow:Assets written to: gs://vertex-ai-demo-351514/export/natality_20220607-221449/assets


Once we have the model artifact stored, we are going use the SDK to upload the model to the Vertex AI registry. 

In [23]:
from google.cloud import aiplatform
my_model = aiplatform.Model.upload(display_name='natality-test'+current_time,
                                  artifact_uri=export_dir,
                                  serving_container_image_uri='us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-8:latest',
                                  project=PROJECT_ID)



Creating Model
Create Model backing LRO: projects/697349885677/locations/us-central1/models/7512976146732941312/operations/8024892886426320896
Model created. Resource name: projects/697349885677/locations/us-central1/models/7512976146732941312
To use this Model in another session:
model = aiplatform.Model('projects/697349885677/locations/us-central1/models/7512976146732941312')


If you go back to the Vertex AI Cloud Console, you will see that you now have created a model, let's deploy it to an endpoint

In [24]:
endpoint = my_model.deploy(
     deployed_model_display_name='my-endpoint',
     traffic_split={"0": 100},
     machine_type="n1-standard-4",
     accelerator_count=0,
     min_replica_count=1,
     max_replica_count=1,
   )

Creating Endpoint
Create Endpoint backing LRO: projects/697349885677/locations/us-central1/endpoints/216621382857916416/operations/310226724740661248
Endpoint created. Resource name: projects/697349885677/locations/us-central1/endpoints/216621382857916416
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/697349885677/locations/us-central1/endpoints/216621382857916416')
Deploying model to Endpoint : projects/697349885677/locations/us-central1/endpoints/216621382857916416
Deploy Endpoint model backing LRO: projects/697349885677/locations/us-central1/endpoints/216621382857916416/operations/6232460234732863488
Endpoint model deployed. Resource name: projects/697349885677/locations/us-central1/endpoints/216621382857916416
